# Damiano Bressanin 138075


Ho preso spunto da:
- https://keras.io/keras_tuner
- https://keras.io/guides/keras_tuner/getting_started/


# Test:
- Dataset 64-16-20;
- Foto RGB;
- No Data Augmentation;
- Random Search per iperparametri con KerasTuner;

In [1]:
import os
import shutil
import random
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

import numpy as np

from matplotlib.image import imread
from matplotlib import pyplot

random.seed(42)

2023-08-14 10:32:55.705148: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Normalizzazione e Data Augmentation

In [3]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
#    shear_range=0.3,
#    zoom_range=0.3,
#    horizontal_flip=True,
    vertical_flip=False,
#    brightness_range=[0.8,1.2],
#    rotation_range=20,
#    width_shift_range=0.15,
#    height_shift_range=0.15,
#    fill_mode='nearest'
)


test_datagen = ImageDataGenerator(rescale=1./255)

val_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 64

train_it = train_datagen.flow_from_directory("dataset_diviso/train/",
                                             class_mode='binary',
                                             batch_size=batch_size,
                                             target_size=(200, 200),
                                             color_mode="rgb",
                                             shuffle=True,
                                             seed=42
                                            )

val_it = val_datagen.flow_from_directory('dataset_diviso/valid/',
                                         class_mode='binary',
                                         batch_size=batch_size,
                                         target_size=(200, 200),
                                         color_mode="rgb",
                                         shuffle=False,
                                         seed=42)

test_it = test_datagen.flow_from_directory('dataset_diviso/test/',
                                           class_mode='binary',
                                           batch_size=batch_size,
                                           target_size=(200, 200),
                                           color_mode="rgb",
                                           shuffle=False,
                                           seed=42)



Found 15999 images belonging to 2 classes.
Found 3998 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


# Creazione del modello usando KerasTuner


In [6]:
import datetime

unique_name = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = "logs/fit/KerasTuner" + unique_name

tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1, update_freq='epoch')

In [7]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [8]:
import keras_tuner
from tensorflow import keras
from keras_tuner.tuners import RandomSearch


def build_model(hp):
    model = Sequential()

    model.add(Conv2D(filters=hp.Int('conv_1_filter', min_value=32, max_value=64, step=16),
                     kernel_size=hp.Choice('conv_1_kernel', values=[3]), #3,5
                     activation='relu',
                     padding='same',
                     input_shape=(200, 200, 3)))
    
    model.add(MaxPooling2D(2, 2))
    
    model.add(Dropout(rate=hp.Float('dropout_1', min_value=0.1, max_value=0.4, step=0.1)))

    
    model.add(Conv2D(filters=hp.Int('conv_2_filter', min_value=64, max_value=128, step=32),
                     kernel_size=hp.Choice('conv_2_kernel', values=[3]),
                     activation='relu',
                     padding='same'))
    
    model.add(MaxPooling2D(2, 2))
    
    model.add(Dropout(rate=hp.Float('dropout_2', min_value=0.1, max_value=0.3, step=0.1)))

    
    model.add(Conv2D(filters=hp.Int('conv_3_filter', min_value=128, max_value=256, step=64),
                     kernel_size=hp.Choice('conv_3_kernel', values=[3]),
                     activation='relu',
                     padding='same'))
    
    model.add(MaxPooling2D(2, 2))
    
    model.add(Dropout(rate=hp.Float('dropout_3', min_value=0.1, max_value=0.3, step=0.1)))
    
    
    model.add(Flatten())

    model.add(Dense(units=hp.Int('dense_1_units', min_value=128, max_value=256, step=32),
                    activation='relu'))
    
    model.add(Dropout(rate=hp.Float('dropout_4', min_value=0.2, max_value=0.4, step=0.1)))
    

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'sgd']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model


tuner = RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=10,
                     executions_per_trial=3,
                     directory='3blockKerasTuner',
                     project_name='canigatti')

# Effettuo la ricerca degli iperparametri
tuner.search(train_it,
             epochs=50,
             validation_data=val_it,
             callbacks=[early_stop, tensorboard_callback])

# 10 combinazioni, ognuna addestrata 3 volte con i pesi inizializzati a valori casuali = 30 addestramenti.
# Ogni addestramento dura 50 epoche ma con early stopping.

# Prendo il miglior modello
best_model = tuner.get_best_models()[0]

Trial 10 Complete [00h 22m 35s]
val_accuracy: 0.7056028048197428

Best val_accuracy So Far: 0.8161580761273702
Total elapsed time: 11h 50m 30s
INFO:tensorflow:Oracle triggered exit


In [9]:

_, acc = best_model.evaluate(test_it, steps=len(test_it), verbose=1)
print('> %.3f' % (acc * 100.0))

79/79 [==============================] - 14s 167ms/step - loss: 0.4207 - accuracy: 0.8078
> 80.780


In [10]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 200, 200, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 100, 100, 64)      0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 100, 100, 64)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 100, 100, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 50, 50, 64)        0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 50, 50, 64)        0

In [11]:
model_dir = "modelli/KerasTuner"+ unique_name +".keras"
best_model.save(model_dir)